In [97]:
import torch
print(torch.__version__)
from math import log

2.1.2


In [98]:
D = 4
H = 8
R0 = 0.1
op = int(1e5)
batch = 1
hidden_size = 100
opt = 0.01

In [99]:
from collections import OrderedDict

model = torch.nn.Sequential(
    OrderedDict([
        ("linear_1", torch.nn.Linear(D * H, hidden_size)),
        ("activation_1", torch.nn.ReLU()),
        ("linear_2", torch.nn.Linear(hidden_size, D + 2))
    ])
)

In [100]:
optimizer = torch.optim.Adam(
    model.parameters(),  # Передаём все тензоры, учавствующие в градиентном спуске
    lr=0.0001,  # learning rate - величина шага градиентного спуска
)


In [101]:
def RIGHT_staaate(state):
    global H, D, batch
    res = [[float(0)] * H * D for i in range(batch)]
    for k in range(0, batch):
        for i in range(0, D):
            res[k][state[k][i] + i * H] = float(1)
    return res

In [102]:
def reward(x):
    global R0, D, H
    f1 = True
    f2 = True
    for d in range(D):
        if not (0.25 < abs(x[d] / (H - 1) - 0.5) <= 0.5): f1 = False
        if not (0.3 < abs(x[d] / (H - 1) - 0.5) < 0.4): f2 = False
    return (R0 + 0.5 * f1 + 2 * f2)


In [103]:
def loss_fn():
    global H, D, batch
    used = [0] * batch
    allstate = [[0] * D for i in range(batch)]
    way = [[]] * batch
    while True:
        pred = model(torch.tensor(RIGHT_staaate(allstate)))
        p_Fs = pred[:, 0: -1]
        Fs = pred[:, -1]
        was = False
        for j in range(0, batch):
            if used[j]:
                continue
            was = True
            for i in range(0, D):
                if allstate[j][i] == H - 1:
                    p_Fs[j][i] = -float("inf")
        if was == False:
            break
        m = torch.nn.Softmax(dim=1)
        p_Fs = m(p_Fs)
        while True:
            ind = torch.distributions.categorical.Categorical(p_Fs).sample()
            iscorrect = True
            for j in range(0, batch):
                if used[j] or ind[j] == D:
                    continue
                if allstate[j][ind[j]] == H - 1:
                    iscorrect = False
            if iscorrect == False:
                print("OPS!")
                continue
            for j in range(0, batch):
                if used[j]:
                    continue
                p_B = 0
                for i in range(0, D):
                    if allstate[j][i] >= 1:
                        p_B += 1
                p_B = max(p_B, 1)
                way[j].append([Fs[j], torch.log(p_Fs[j][ind[j]]), log(1 / p_B)])
                if ind[j] == D:
                    #way[j][-1][1] = 1
                    way.append([log(reward(allstate[j])), 0, 0])
                    used[j] = True
                else:
                    assert(allstate[j][ind[j]] != H - 1)
                    allstate[j][ind[j]] += 1
            break
    j = 0
    loss = torch.tensor(0.0)
    for state in allstate:
        for i in range(0, len(way[j]) - 1):
            loss += (way[j][i][0] + way[j][i][1] - way[j][i + 1][0] - way[j][i + 1][2]) ** 2
        j += 1
    loss /= batch
    return loss, allstate

In [104]:
from itertools import product

rewards = torch.zeros(*[H for i in range(D)])
coord_diap = [range(H) for _ in range(D)]

for coord in product(*coord_diap):
    rewards[tuple(coord)] = reward(torch.tensor(coord))

rewards /= rewards.sum()

def loss2(a, samples=1000):
    counter = torch.zeros(*[H for i in range(D)])
    for i in range(samples):
        counter[tuple(a[i])] += 1

    counter /= counter.sum()

    return (rewards - counter).abs().sum()


In [105]:
a = []
for i in range(op):
    optimizer.zero_grad()
    loss = 0
    x = loss_fn()
    loss = x[0]
    print(loss)
    for j in x[1]:
        a.append(j)
    loss.backward()
    optimizer.step()
    if i % 100 == 0:
        print(i * batch)
        assert(torch.tensor(a[-10000:]).max() <= 7)
        print(loss2(a[-10000:], len(a[-10000:])))

tensor(3.9072, grad_fn=<DivBackward0>)
0
tensor(1.9979)
tensor(4.1165, grad_fn=<DivBackward0>)
tensor(4.5479, grad_fn=<DivBackward0>)
tensor(4.1908, grad_fn=<DivBackward0>)
tensor(9.4090, grad_fn=<DivBackward0>)
tensor(5.3754, grad_fn=<DivBackward0>)
tensor(6.5991, grad_fn=<DivBackward0>)
tensor(5.6392, grad_fn=<DivBackward0>)
tensor(0.)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn